In [ ]:
import numpy as np
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

# Load and preprocess the MNIST dataset
(train_images, _), (_, _) = mnist.load_data()
train_images = train_images[:10000]
train_images = train_images.astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize between -1 and 1
train_images = expand_dims(train_images, axis=-1)

# Define the standalone discriminator model
def define_discriminator(in_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7, 7), activation='tanh', padding='same'))
    return model

# Define the GAN model
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Generate real samples from the MNIST dataset
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y

# Generate random points in the latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# Generate fake samples using the generator
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

# Train the GAN
def train_gan(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=100, n_batch=64):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss1, _ = discriminator.train_on_batch(X_real, y_real)
            d_loss2, _ = discriminator.train_on_batch(X_fake, y_fake)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print(f"Epoch {i+1}, Batch {j+1}/{bat_per_epo}, D1={d_loss1:.3f}, D2={d_loss2:.3f}, G={g_loss:.3f}")
        if (i+1) % 5 == 0:
            summarize_performance(i, generator, discriminator, latent_dim)

# Plot generated images during training
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    x_real, y_real = generate_real_samples(train_images, n)
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    print(f"Epoch {epoch+1}: Real Accuracy = {acc_real:.4f}, Fake Accuracy = {acc_fake:.4f}")

    plt.figure(figsize=(10, 10))
    for i in range(100):
        plt.subplot(10, 10, 1 + i)
        plt.axis('off')
        plt.imshow(x_fake[i, :, :, 0], cmap='gray_r')
    plt.savefig(f'generated_plot_epoch_{epoch+1}.png')
    plt.close()

# Size of the latent space
latent_dim = 5

# Create the discriminator
discriminator = define_discriminator()

# Create the generator
generator = define_generator(latent_dim)

# Create the GAN
gan_model = define_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan_model, train_images, latent_dim, n_epochs=100, n_batch=64)


1/1 [==============================] - 0s 68ms/step
Epoch 1, Batch 1/156, D1=0.646, D2=0.702, G=0.689
1/1 [==============================] - 0s 47ms/step
Epoch 1, Batch 2/156, D1=0.308, D2=0.739, G=0.653
1/1 [==============================] - 0s 48ms/step
Epoch 1, Batch 3/156, D1=0.124, D2=0.932, G=0.537
1/1 [==============================] - 0s 47ms/step
Epoch 1, Batch 4/156, D1=0.079, D2=1.381, G=0.432
1/1 [==============================] - 0s 31ms/step
Epoch 1, Batch 5/156, D1=0.154, D2=1.364, G=0.548
1/1 [==============================] - 0s 31ms/step
Epoch 1, Batch 6/156, D1=0.318, D2=0.875, G=0.770
1/1 [==============================] - 0s 47ms/step
Epoch 1, Batch 7/156, D1=0.444, D2=0.631, G=1.020
1/1 [==============================] - 0s 47ms/step
Epoch 1, Batch 8/156, D1=0.444, D2=0.477, G=1.312
1/1 [==============================] - 0s 31ms/step
Epoch 1, Batch 9/156, D1=0.478, D2=0.358, G=1.516
1/1 [==============================] - 0s 31ms/step
Epoch 1, Batch 10/156, D1=0.34